In [1]:
import geopandas as gpd
import os
import pandas as pd
import folium
import dotenv
from folium.plugins import MarkerCluster, Search
import numpy as np
import requests

dotenv.load_dotenv()
NAVER_CLINET_ID = os.getenv("NAVER_SERACH_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_SERACH_SECRET")

In [2]:
def merge_csv_into_gdf():
    gdf = None
    for (dir,_,file) in os.walk("./data/"):
        print(file)
        for f in file:
            if gdf is None:
                gdf = pd.read_csv(dir+f)
            else:
                gdf= pd.concat([gdf,pd.read_csv(dir+f)],ignore_index=True)

    gdf = gpd.GeoDataFrame(gdf,geometry=gpd.points_from_xy(gdf.lng,gdf.lat),crs="epsg:4326")
    return gdf

In [3]:
all_gdf = merge_csv_into_gdf()
all_gdf

['traditional_w_coordinate_27500-27600.csv', 'traditional_w_coordinate_15100-15200.csv', 'traditional_w_coordinate_13900-14000.csv', 'traditional_w_coordinate_1700-1800.csv', 'traditional_w_coordinate_14500-14600.csv', 'traditional_w_coordinate_18300-18400.csv', 'traditional_w_coordinate_1100-1200.csv', 'traditional_w_coordinate_26100-26200.csv', 'traditional_w_coordinate_7600-7700.csv', 'traditional_w_coordinate_5000-5100.csv', 'traditional_w_coordinate_3900-4000.csv', 'traditional_w_coordinate_8000-8100.csv', 'traditional_w_coordinate_1800-1900.csv', 'traditional_w_coordinate_10100-10200.csv', 'traditional_w_coordinate_19400-19500.csv', 'traditional_w_coordinate_22500-22600.csv', 'traditional_w_coordinate_23100-23200.csv', 'traditional_w_coordinate_7700-7800.csv', 'traditional_w_coordinate_18000-18100.csv', 'traditional_w_coordinate_11500-11600.csv', 'traditional_w_coordinate_1600-1700.csv', 'traditional_w_coordinate_7100-7200.csv', 'traditional_w_coordinate_3000-3100.csv', 'traditio

,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry
0,28876,319,화양제일골목시장(화양제일시장),화양식품,서울특별시 광진구 아차산로29길 59(화양동)1층,02-465-6374,두부,37.543482,127.069282,POINT (127.06928 37.54348)
1,28877,318,화양제일골목시장(화양제일시장),레드크라운,서울특별시 광진구 아차산로29길 56(화양동)1층,02-3409-5545,도소매(케이스),37.543187,127.069398,POINT (127.06940 37.54319)
2,28878,317,화양제일골목시장(화양제일시장),해안수산,서울 광진구 동일로24길 70(화양동),NaN,농축수산품,37.542833,127.068790,POINT (127.06879 37.54283)
3,28879,316,화양제일골목시장(화양제일시장),신토불이청과,서울특별시 광진구 동일로24길 64(화양동)1층,02-463-6740,농산물,37.542940,127.068531,POINT (127.06853 37.54294)
4,28880,315,화양제일골목시장(화양제일시장),제일상회,서울특별시 광진구 아차산로29길 59(화양동) 1층,02-465-3067,농산물,37.543482,127.069282,POINT (127.06928 37.54348)
...,...,...,...,...,...,...,...,...,...,...
27803,17530,11124,성내전통시장,행운축산,서울특별시 강동구 풍성로37가길 17(성내동) 1층,02-489-5221,소매(식육),37.533297,127.127313,POINT (127.12731 37.53330)
27804,17531,11123,성내전통시장,속옷할인매장,서울특별시 강동구 천호대로162길 50(성내동) 속옷할인매장,NaN,소매(속옷),37.534700,127.127870,POINT (127.12787 37.53470)
27805,17532,11122,성내전통시장,김치전문,서울특별시 강동구 천호대로162길 73(성내동)1층 3호,NaN,소매(김치),37.533393,127.127705,POINT (127.12770 37.53339)
27806,17533,11121,성내전통시장,안성꽃상회,서울특별시 강동구 천호대로162길 65(성내동) (성내골목시장)안성꽃상회 1층 102호,02-484-2581,생화 양곡,37.533990,127.127835,POINT (127.12784 37.53399)


# Filter data : fiiled vs empty

In [4]:
data_with_coord = all_gdf[all_gdf["lat"].notna()]
data_wo_coord = all_gdf[all_gdf["lat"].isnull()]
data_wo_coord.to_csv("./data_wo_coord.csv")


# Define function for request to naver search api

In [5]:
def request_to_naver_local_api(row):
    base_url = "https://openapi.naver.com/v1/search/local.json"
    if isinstance(row["주소"],str) is False:
        return
    query_text = row["상호명"]+" ,"+" ".join(row["주소"].split()[:3])
    params = dict()
    headers = dict()
    params["query"] = query_text
    headers["X-Naver-Client-Id"] = NAVER_CLINET_ID
    headers["X-Naver-Client-Secret"] = NAVER_CLIENT_SECRET
    print(params)
    r= requests.get(base_url,params=params,headers=headers)
    print(r.status_code)
    if r.status_code == 200:
        res_json = r.json()['items']
        print(res_json)
        if len(res_json) < 1:
            return
        return pd.Series([res_json[0]["mapx"],
                         res_json[0]["mapy"]]
                        )
    else:
        return


# Validate request code with test df

In [6]:
test_df = data_wo_coord[1:10]
test_df

,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry
16,28892,303,황학동 주방가구거리 상점가,건영주방,서울 중구 마장로 1011층 연풍주차장 (황학동),NaN,주방용식기,NaN,NaN,POINT EMPTY
17,28893,302,황학동 주방가구거리 상점가,모아종합주방,서울 중구 마장로 1081층(황학동),NaN,주방용식기,NaN,NaN,POINT EMPTY
19,28895,300,황학동 주방가구거리 상점가,세한종합주방,서울 중구 마장로 791층 (황학동),NaN,주방용식기,NaN,NaN,POINT EMPTY
21,28898,298,황학동 주방가구거리 상점가,프렌드주방시스템,서울 중구 마장로 991층 (황학동),NaN,주방용식기,NaN,NaN,POINT EMPTY
45,28923,274,황학동 주방가구거리 상점가,신대경종합주방,서울 중구 마장로 681층 3층 (흥인동),NaN,주방용구,NaN,NaN,POINT EMPTY
56,28934,263,황학동 주방가구거리 상점가,태흥싱크,서울 중구 마장로 871층 (황학동),NaN,주방용구,NaN,NaN,POINT EMPTY
57,28935,262,황학동 주방가구거리 상점가,티제이(TJ)주방,"서울 중구 마장로 701,2층 (흥인동)",NaN,주방용구,NaN,NaN,POINT EMPTY
62,28941,257,황학동 주방가구거리 상점가,호반종합상사,서울 중구 마장로 721층(흥인동),NaN,주방용구,NaN,NaN,POINT EMPTY
65,28944,254,황학동 주방가구거리 상점가,대성사,서울 중구 마장로 87112-2호 신당지하도상가 (황학동),NaN,정장,NaN,NaN,POINT EMPTY


In [7]:
test_df.apply(request_to_naver_local_api,axis=1)

{'query': '건영주방 ,서울 중구 마장로'}
200
[{'title': '<b>건영주방</b>그릇', 'link': '', 'category': '쇼핑,유통>주방용품', 'description': '', 'telephone': '', 'address': '서울특별시 중구 황학동 1205', 'roadAddress': '서울특별시 중구 마장로 89', 'mapx': '1270198965', 'mapy': '375684684'}]
{'query': '모아종합주방 ,서울 중구 마장로'}
200
[{'title': '<b>모아종합주방</b>', 'link': 'http://모아주방.com', 'category': '쇼핑,유통>주방용품', 'description': '', 'telephone': '', 'address': '서울특별시 중구 황학동 1184', 'roadAddress': '서울특별시 중구 마장로 108', 'mapx': '1270216332', 'mapy': '375680564'}]
{'query': '세한종합주방 ,서울 중구 마장로'}
200
[{'title': '<b>세한종합주방</b>', 'link': 'http://www.jubang119.com/', 'category': '쇼핑,유통>주방용품', 'description': '', 'telephone': '', 'address': '서울특별시 중구 황학동 123', 'roadAddress': '서울특별시 중구 마장로 79 삼천리종합주방', 'mapx': '1270184730', 'mapy': '375684880'}]
{'query': '프렌드주방시스템 ,서울 중구 마장로'}
200
[]
{'query': '신대경종합주방 ,서울 중구 마장로'}
200
[{'title': '<b>신대경종합주방</b>', 'link': '', 'category': '쇼핑,유통>주방용품', 'description': '', 'telephone': '', 'address': '서울특별시 중구 흥인동 2-23', 'r

,0,1
16,1270198965,375684684
17,1270216332,375680564
19,1270184730,375684880
21,None,None
45,1270172820,375680394
56,None,None
57,1270176438,375680718
62,1270177329,375680845
65,None,None


#Apply

In [8]:
data_wo_coord[['lng','lat']] =data_wo_coord.apply(request_to_naver_local_api,axis=1)
lat_lng_filled = data_wo_coord.copy()
lat_lng_filled

{'query': '씨유(CU) 신당메트하임점 ,서울 중구 마장로'}
200
[{'title': '<b>CU</b> <b>신당메트하임점</b>', 'link': 'https://twitter.com/BGFretail', 'category': '생활,편의>편의점', 'description': '', 'telephone': '', 'address': '서울특별시 중구 황학동 1074-2', 'roadAddress': '서울특별시 중구 마장로 116 (황학동)', 'mapx': '1270227031', 'mapy': '375678794'}]
{'query': '건영주방 ,서울 중구 마장로'}
200
[{'title': '<b>건영주방</b>그릇', 'link': '', 'category': '쇼핑,유통>주방용품', 'description': '', 'telephone': '', 'address': '서울특별시 중구 황학동 1205', 'roadAddress': '서울특별시 중구 마장로 89', 'mapx': '1270198965', 'mapy': '375684684'}]
{'query': '모아종합주방 ,서울 중구 마장로'}
200
[{'title': '<b>모아종합주방</b>', 'link': 'http://모아주방.com', 'category': '쇼핑,유통>주방용품', 'description': '', 'telephone': '', 'address': '서울특별시 중구 황학동 1184', 'roadAddress': '서울특별시 중구 마장로 108', 'mapx': '1270216332', 'mapy': '375680564'}]
{'query': '세한종합주방 ,서울 중구 마장로'}
200
[{'title': '<b>세한종합주방</b>', 'link': 'http://www.jubang119.com/', 'category': '쇼핑,유통>주방용품', 'description': '', 'telephone': '', 'address': '서울특별시 중구 황학동 12

/opt/homebrew/anaconda3/envs/gis/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry
11,28887,308,황학동 주방가구거리 상점가,씨유(CU) 신당메트하임점,"서울 중구 마장로 1161층 101호(황학동, 신당메트하임오피스텔)",NaN,편의점,375678794,1270227031,POINT EMPTY
16,28892,303,황학동 주방가구거리 상점가,건영주방,서울 중구 마장로 1011층 연풍주차장 (황학동),NaN,주방용식기,375684684,1270198965,POINT EMPTY
17,28893,302,황학동 주방가구거리 상점가,모아종합주방,서울 중구 마장로 1081층(황학동),NaN,주방용식기,375680564,1270216332,POINT EMPTY
19,28895,300,황학동 주방가구거리 상점가,세한종합주방,서울 중구 마장로 791층 (황학동),NaN,주방용식기,375684880,1270184730,POINT EMPTY
21,28898,298,황학동 주방가구거리 상점가,프렌드주방시스템,서울 중구 마장로 991층 (황학동),NaN,주방용식기,None,None,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...
27744,17468,11183,선유도역골목형상점가,앙앤온,서울 영등포구 양평로19길 36층 (양평동4가),NaN,기타음료식품,375368344,1268950670,POINT EMPTY
27745,17469,11182,선유도역골목형상점가,원띵 비헬스,"서울 영등포구 선유로49길 236층 603호 (양평동4가, 선유도역2차아이에스비즈타워)",NaN,기타건강식,None,None,POINT EMPTY
27749,17474,11178,성내전통시장,웰스마켓,"서울 강동구 천호대로162길 7310층 1002호 (성내동, 원다빌2)",NaN,홍삼제품,None,None,POINT EMPTY
27758,17483,11169,성내전통시장,강동 정육식당,서울 강동구 천호대로162길 671층 (성내동),NaN,일반한식,375338736,1271277966,POINT EMPTY


In [9]:
lat_lng_filled.to_csv("./filled_by_naver.csv")